In [1]:
# check node state
! nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address       Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  192.168.32.2  70.26 KiB  16      59.3%             5b7b2abb-114c-46e3-a493-fa5ca99abf70  rack1
UN  192.168.32.4  70.26 KiB  16      79.1%             1314dc75-070b-446a-9319-51de3522cf52  rack1
UN  192.168.32.3  70.26 KiB  16      61.6%             d5941574-0702-4849-8b4a-079d8e5607b7  rack1



In [2]:
# create keyspace(database) and import data
from cassandra.cluster import Cluster
cluster = Cluster(['p6-db-1', 'p6-db-2', 'p6-db-3'])
cass = cluster.connect()
cass.execute('drop keyspace if exists weather')
cass.execute("create keyspace weather with replication={'class': 'SimpleStrategy', 'replication_factor': 3}")
cass.execute('use weather')
cass.execute('''
create type station_record(
    tmin INT,
    tmax INT
)
''')
cass.execute('''
create table stations(
    id TEXT,
    name TEXT STATIC,
    date DATE,
    record weather.station_record,
    PRIMARY KEY((id), date)
) WITH CLUSTERING ORDER BY (date ASC)
''')

In [3]:
#q1
cass.execute('describe table weather.stations').one().create_statement

"CREATE TABLE weather.stations (\n    id text,\n    date date,\n    name text static,\n    record station_record,\n    PRIMARY KEY (id, date)\n) WITH CLUSTERING ORDER BY (date ASC)\n    AND additional_write_policy = '99p'\n    AND bloom_filter_fp_chance = 0.01\n    AND caching = {'keys': 'ALL', 'rows_per_partition': 'NONE'}\n    AND cdc = false\n    AND comment = ''\n    AND compaction = {'class': 'org.apache.cassandra.db.compaction.SizeTieredCompactionStrategy', 'max_threshold': '32', 'min_threshold': '4'}\n    AND compression = {'chunk_length_in_kb': '16', 'class': 'org.apache.cassandra.io.compress.LZ4Compressor'}\n    AND memtable = 'default'\n    AND crc_check_chance = 1.0\n    AND default_time_to_live = 0\n    AND extensions = {}\n    AND gc_grace_seconds = 864000\n    AND max_index_interval = 2048\n    AND memtable_flush_period_in_ms = 0\n    AND min_index_interval = 128\n    AND read_repair = 'BLOCKING'\n    AND speculative_retry = '99p';"

In [4]:
# get 
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName("p6")
         .config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.0')
         .config("spark.sql.extensions", "com.datastax.spark.connector.CassandraSparkExtensions")
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1dd11bea-3914-47bb-9281-316713ad292f;1.0
	confs: [default]


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central


	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central


	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central


...

	[SUCCESSFUL ] org.scala-lang#scala-reflect;2.12.11!scala-reflect.jar (160ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/native-protocol/1.5.0/native-protocol-1.5.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#native-protocol;1.5.0!native-protocol.jar(bundle) (37ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-shaded-guava/25.1-jre-graal-sub-1/java-driver-shaded-guava-25.1-jre-graal-sub-1.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1!java-driver-shaded-guava.jar (156ms)


downloading https://repo1.maven.org/maven2/com/typesafe/config/1.4.1/config-1.4.1.jar ...
	[SUCCESSFUL ] com.typesafe#config;1.4.1!config.jar(bundle) (29ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.26/slf4j-api-1.7.26.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.26!slf4j-api.jar (25ms)
downloading https://repo1.maven.org/maven2/io/dropwizard/metrics/metrics-core/4.1.18/metrics-core-4.1.18.jar ...
	[SUCCESSFUL ] io.dropwizard.metrics#metrics-core;4.1.18!metrics-core.jar(bundle) (26ms)
downloading https://repo1.maven.org/maven2/org/hdrhistogram/HdrHistogram/2.1.12/HdrHistogram-2.1.12.jar ...
	[SUCCESSFUL ] org.hdrhistogram#HdrHistogram;2.1.12!HdrHistogram.jar(bundle) (27ms)
downloading https://repo1.maven.org/maven2/org/reactivestreams/reactive-streams/1.0.3/reactive-streams-1.0.3.jar ...
	[SUCCESSFUL ] org.reactivestreams#reactive-streams;1.0.3!reactive-streams.jar (27ms)
downloading https://repo1.maven.org/maven2/com/github/stephenc/jcip/jcip-annotations/1.0

	[SUCCESSFUL ] com.google.code.findbugs#jsr305;3.0.2!jsr305.jar (25ms)
downloading https://repo1.maven.org/maven2/com/datastax/oss/java-driver-query-builder/4.13.0/java-driver-query-builder-4.13.0.jar ...
	[SUCCESSFUL ] com.datastax.oss#java-driver-query-builder;4.13.0!java-driver-query-builder.jar(bundle) (31ms)
:: resolution report :: resolve 3427ms :: artifacts dl 1368ms
	:: modules in use:
	com.datastax.oss#java-driver-core-shaded;4.13.0 from central in [default]
	com.datastax.oss#java-driver-mapper-runtime;4.13.0 from central in [default]
	com.datastax.oss#java-driver-query-builder;4.13.0 from central in [default]
	com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 from central in [default]
	com.datastax.oss#native-protocol;1.5.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 from central in [default]
	com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 from central in [default]
	com.github.spotbugs#spotbugs-annotations;3.

23/11/22 20:33:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
from pyspark.sql.functions import col, expr, lower
df = spark.read.text('ghcnd-stations.txt')
df = df.withColumn('ID', expr('substring (value,1,11)'))
df = df.withColumn('STATE', expr('substring (value,39,2)'))
df = df.withColumn('NAME', expr('substring (value,42,30)'))

In [6]:
#df2.select('state').limit(10).filter(lower(col('state')).like("%st johns%")).toPandas()
df = df.filter(lower(col('state')).like('%wi%'))

In [7]:
# check if row is 1313
df.toPandas()

,value,ID,STATE,NAME
0,US1WIAD0002 43.9544 -89.8096 294.4 WI ADAMS...,US1WIAD0002,WI,ADAMS 0.4 E
1,US1WIAD0005 44.2053 -89.8480 305.7 WI NEKOO...,US1WIAD0005,WI,NEKOOSA 8.0 SSE
2,US1WIAD0006 43.8858 -89.7259 307.8 WI GRAND...,US1WIAD0006,WI,GRAND MARSH 1.0 W
3,US1WIAD0008 43.8611 -89.7163 310.0 WI GRAND...,US1WIAD0008,WI,GRAND MARSH 1.9 SSW
4,US1WIAD0010 43.7864 -89.6417 293.8 WI OXFOR...,US1WIAD0010,WI,OXFORD 4.0 W
...,...,...,...,...
1308,USW00094930 43.9333 -90.2667 280.1 WI VOLK ...,USW00094930,WI,VOLK FLD ANG
1309,USW00094940 43.9667 -90.7333 252.7 WI SPART...,USW00094940,WI,SPARTA FT MCCOY
1310,USW00094973 46.0303 -91.4425 368.8 WI HAYWA...,USW00094973,WI,HAYWARD MUNI AP
1311,USW00094985 44.6378 -90.1875 381.6 WI MARSH...,USW00094985,WI,MARSHFIELD MUNI AP


In [8]:
insert_data = df.collect()
len(insert_data)

1313

In [9]:
# insert data into cassandra weather database stations table
insert_cql = cass.prepare('''
    insert into weather.stations (id, name)
    values (?, ?)
''')
for i in insert_data:
    cass.execute(insert_cql, (i['ID'], i['NAME']))

In [10]:
# check if row of data in cassandra is 1313
list(cass.execute('select count(*) from weather.stations;'))

[Row(count=1313)]

In [11]:
#q2
cass.execute("select name from weather.stations where id = 'USW00014837'").one()[0]

'MADISON DANE CO RGNL AP       '

In [12]:
#q3
tokenid = cass.execute("select token(id) from weather.stations where id = 'USC00470273'").one()[0]
tokenid

-9014250178872933741

In [13]:
# get the output of command "nodetool ring"
import subprocess
result = subprocess.run("nodetool ring", shell=True, stdout=subprocess.PIPE, text=True)
result = result.stdout.strip().split('\n')

In [14]:
# get the vnode number 
tokens = []
for s in result:
    try:
        tokens.append(int(s.strip().split(' ')[-1]))
    except Exception as e:
        pass

In [15]:
#q4
# output the vnode number that is bigger than my token id and break
ans = tokens[0]
for t in tokens[1:]:
    if t >= tokenid:
        ans = t
        break
ans

-8795293329753344364

In [16]:
#! unzip records.zip

In [17]:
# read parquetfile, do pivot to make a column into several columns (for tmin, tmax), and change the date format
from pyspark.sql.functions import col, collect_list
from pyspark.sql.functions import date_format, to_date
df = spark.read.parquet('records.parquet')
df = df.groupby('station','date').pivot("element").agg({"value": "first"})
df = df.withColumnRenamed('TMIN', 'tmin').withColumnRenamed('TMAX', 'tmax')
df = df.withColumn("date", date_format(to_date(col("date"), "yyyyMMdd"), "yyyy-MM-dd"))
records = df.collect()

23/11/22 20:34:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [18]:
# ! python3 server.py (run the grpc sercer)

In [19]:
# keep sending data to the grpc server and let the server upload my data to cassandra
import station_pb2, station_pb2_grpc
import grpc

port = 5440
channel = grpc.insecure_channel("localhost:" + str(port))
stub = station_pb2_grpc.StationStub(channel)

for i in records:
    args = [i['station'], i['date'], i['tmin'], i['tmax']]
    record_station = i['station']
    record_date = i['date']
    record_tmin = int(i['tmin'])
    record_tmax = int(i['tmax'])
    stub.RecordTemps(station_pb2.RecordTempsRequest(station = record_station, date = record_date, tmin = record_tmin, tmax = record_tmax ))

In [20]:
cass.execute("select max(record.tmax) from weather.stations where id = 'USW00014837'").one()[0]

356

In [21]:
#q5
ans = stub.StationMax(station_pb2.StationMaxRequest(station = 'USW00014837'))
ans.tmax

356

In [22]:
#q6
mydata = (spark.read.format("org.apache.spark.sql.cassandra")
.option("spark.cassandra.connection.host", "p6-db-1,p6-db-2,p6-db-3")
.option("keyspace", "weather")
.option("table", "stations")
.load())
mydata.createOrReplaceTempView('stations')
spark.catalog.listTables()

[Table(name='stations', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [23]:
#q7
ans = spark.sql('select id, avg(record.tmax - record.tmin) from stations where record is not null group by id').collect()
mydict = {}
for i in ans:
    mydict[i['id']] = i['avg((record.tmax - record.tmin))']
mydict

{'USW00014839': 89.6986301369863,
 'USR0000WDDG': 102.06849315068493,
 'USW00014837': 105.62739726027397,
 'USW00014898': 102.93698630136986}

In [24]:
#! kill p6-db-2

In [25]:
#q8
! nodetool status

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address       Load       Tokens  Owns (effective)  Host ID                               Rack 
DN  192.168.32.2  87.74 KiB  16      100.0%            5b7b2abb-114c-46e3-a493-fa5ca99abf70  rack1
UN  192.168.32.4  87.74 KiB  16      100.0%            1314dc75-070b-446a-9319-51de3522cf52  rack1
UN  192.168.32.3  87.74 KiB  16      100.0%            d5941574-0702-4849-8b4a-079d8e5607b7  rack1



In [26]:
#q9
stub.StationMax(station_pb2.StationMaxRequest(station = 'USW00014837'))

error: "need 3 replicas, but only have 2"

In [27]:
#q10
stub.RecordTemps(station_pb2.RecordTempsRequest(station = 'USW00014837', date = '2022-11-22', tmin = 1, tmax = 90 ))